In [9]:
import os
import requests
import pandas as pd
import numpy as np

alphavantage_api_key = open("alphavantage_api_key.txt").read()
ticker = "panw".upper()

# Calls the API to get historical stock data
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&outputsize=full&apikey={alphavantage_api_key}'
r = requests.get(url)
data = r.json()

# Calls the API to get earnings data
url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={alphavantage_api_key}'
r = requests.get(url)
earnings = r.json()

# Yahoo Finance API to get earnings dates
#yfinance_earnings = yf.Ticker(f'{ticker}').get_earnings_dates(limit=30)


In [10]:
##############
# Stock Data #
##############

# Extract the time series data
time_series = data['Time Series (Daily)']

# Convert the time series data into a DataFrame
df = pd.DataFrame.from_dict(time_series, orient='index')

# Convert the index to datetime
df.index = pd.to_datetime(df.index)

# Convert the columns to numeric
df = df.apply(pd.to_numeric)

df = df.reset_index().rename(columns={'index': 'tradingDate'})

#################
# Earnings Data #
#################

# Extract annual earnings data
annual_earnings = earnings['annualEarnings']

# Convert annual earnings data into a DataFrame
annual_earnings_df = pd.DataFrame(annual_earnings)

# Extract quarterly earnings data
quarterly_earnings = earnings['quarterlyEarnings']

# Convert quarterly earnings data into a DataFrame
quarterly_earnings_df = pd.DataFrame(quarterly_earnings)

last_earnings = quarterly_earnings_df[["reportedDate","reportTime"]].head(30)

# Function to adjust the date based on reportTime
def adjust_date(row):
   if row['reportTime'] == 'post-market':
       return row['reportedDate'] + pd.Timedelta(days=1)
   elif row['reportTime'] == 'pre-market':
       return row['reportedDate'] - pd.Timedelta(days=1)
   else:
       return row['reportedDate']

last_earnings['reportedDate'] = pd.to_datetime(last_earnings['reportedDate'])

# Apply the function to create a new column with adjusted dates
# Last earnings is used to determine how the stock moved historically around earnings
last_earnings['adjustedDate'] = last_earnings.apply(adjust_date, axis=1)

#############
# merged_df #
#############

#merged_df joins the first set of earnings data
merged_df = pd.merge(last_earnings, df, left_on='reportedDate', right_on="tradingDate", how='left')
columns_to_drop = ['tradingDate','5. volume']
merged_df.drop(columns=columns_to_drop, inplace=True)
merged_df = merged_df.rename(columns={
   '1. open': 'report open',
   '2. high': 'report high',
   '3. low': 'report low',
   '4. close': 'report close'
})

###############
# merged_2_df #
###############

#merged_2_df is the final joined dataset that combines the first set of earnings data with the second set of earnings data
merged_2_df = pd.merge(merged_df, df, left_on='adjustedDate', right_on="tradingDate", how='left')

merged_2_df.drop(columns=columns_to_drop, inplace=True)

merged_2_df = merged_2_df.rename(columns={
   '1. open': 'adjusted open',
   '2. high': 'adjusted high',
   '3. low': 'adjusted low',
   '4. close': 'adjusted close'
})

################
# calculations #
################

# used to calculate the difference between the high and low depending is earnings was pre-market or post-market
# and if the stock went up or down

# Initialize a new column with NaN values to calculate the difference between high and low
merged_2_df['diff'] = np.nan

# Iterate over each row and apply the conditions
for index, row in merged_2_df.iterrows():
   #Post-market
   if row['reportTime'] == 'post-market':
       if row['report close'] > row['adjusted open']:
           merged_2_df.at[index, 'diff'] = round(row['adjusted low'] - row['report high'],2)
       elif row['report close'] < row['adjusted open']:
           merged_2_df.at[index, 'diff'] = round(row['adjusted high'] - row['report low'],2)
       else:
           merged_2_df.at[index, 'diff'] = np.nan
   #Pre-market
   elif row['reportTime'] == 'pre-market':
       if row['adjusted close'] > row['report open']:
           merged_2_df.at[index, 'diff'] = round(row['report low'] - row['adjusted high'],2)
       elif row['adjusted close'] < row['report open']:
           merged_2_df.at[index, 'diff'] = round(row['report high'] - row['adjusted low'],2)
       else:
           merged_2_df.at[index, 'diff'] = np.nan

# Initialize a new column with NaN values for the percentage calculation
merged_2_df['percent_diff'] = np.nan

# Iterate over each row and apply the conditions
for index, row in merged_2_df.iterrows():
   if row['reportTime'] == 'post-market':
       if row['report close'] > row['adjusted open']:
           merged_2_df.at[index, 'percent_diff'] = round((row['adjusted low'] / row['report high'])-1,3)
       elif row['report close'] < row['adjusted open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['adjusted high'] / row['report low']-1,3)
       else:
           merged_2_df.at[index, 'percent_diff'] = np.nan
   elif row['reportTime'] == 'pre-market':
       if row['adjusted close'] > row['report open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['report low'] / row['adjusted high']-1,3)
       elif row['adjusted close'] < row['report open']:
           merged_2_df.at[index, 'percent_diff'] = round(row['report high'] / row['adjusted low']-1,3)
       else:
           merged_2_df.at[index, 'percent_diff'] = np.nan

merged_2_df.sort_values(by="percent_diff", ascending=False)


,reportedDate,reportTime,adjustedDate,report open,report high,report low,report close,adjusted open,adjusted high,adjusted low,adjusted close,diff,percent_diff
16,2021-08-23,post-market,2021-08-24,368.7100,374.1300,367.2100,372.57,423.650,446.9093,419.4000,441.87,79.70,0.217
13,2022-05-19,post-market,2022-05-20,427.4400,444.4300,421.5482,436.37,488.040,492.9694,460.4418,478.68,71.42,0.169
12,2022-08-22,post-market,2022-08-23,506.1200,514.3800,500.6111,508.05,559.520,572.6000,552.0000,569.51,71.99,0.144
10,2023-02-21,post-market,2023-02-22,168.5000,169.2000,165.3600,166.89,182.375,188.7885,182.3000,187.75,23.43,0.142
11,2022-11-17,post-market,2022-11-18,156.0000,157.5125,151.4900,156.56,171.275,173.0000,166.0700,167.48,21.51,0.142
26,2019-02-26,post-market,2019-02-27,232.5600,236.0700,231.7300,235.55,258.040,260.6300,250.6000,254.88,28.90,0.125
4,2024-08-19,post-market,2024-08-20,336.0000,344.4778,335.1100,343.36,348.690,375.3699,348.0000,368.01,40.26,0.120
24,2019-09-04,post-market,2019-09-05,200.0000,201.8000,196.2900,200.49,218.020,218.0200,205.1100,212.05,21.73,0.111
27,2018-11-29,post-market,2018-11-30,176.0000,179.3900,173.9500,176.05,184.450,192.0000,171.2600,172.95,18.05,0.104
9,2023-05-23,post-market,2023-05-24,191.5600,193.2000,188.8300,189.74,196.230,207.3799,194.0800,204.31,18.55,0.098
